The goal of this notebook is the replicate an existing Curve pool (stableswap).

In [ ]:
import sys, os
sys.path.append("../../")

from utils.contract_utils import init_contract
from utils.network_utils import configure_network

import brownie
from brownie.network import chain

configure mainnet to alchemy

In [ ]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']

In [ ]:
configure_network(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

In [ ]:
brownie.network.connect('mainnet-fork')

Load pool contract:

In [ ]:
pool_contract_addr = "0x9D0464996170c6B9e75eED71c68B99dDEDf279e8"  # cvxcrv pool addr

pool_contract = init_contract(contract_addr=pool_contract_addr)
pool_contract.info()

In [ ]:
initial_A = pool_contract.initial_A()
initial_A_time = pool_contract.initial_A_time()
A = pool_contract.A()
A_precise = pool_contract.A_precise()
A_precision = int(A_precise/A)

print(f"pool param details:")
print(f"initial_A: {initial_A}")
print(f"initial_A_time: {initial_A_time}")
print(f"A: {A}")
print(f"A_precise: {A_precise}")
print(f"A_precision: {A_precision}")

## Ramp pool to a different A.

Ramp parameters:

In [ ]:
future_A = int(100 * A_precision)
t0 = chain.time()
t1 = int(t0 + 7 * 86400)

Get Admin address:


In [ ]:
admin_addr = "0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a"

Ramp pool:

In [ ]:
print(f"Ramp parameters: _future_A: {future_A}, _future_time: {t1}, msg.sender: {admin_addr}, pool_contract: {pool_contract_addr}")
pool_contract.ramp_A(future_A, t1, {"from": admin_addr})

Sleep for some time and then mine the chain to get new state.

In [ ]:
chain.sleep(t1)
chain.mine()

Check if ramp was successful:

In [ ]:
initial_A = pool_contract.initial_A()
initial_A_time = pool_contract.initial_A_time()
A = pool_contract.A()
A_precise = pool_contract.A_precise()
A_precision = int(A_precise/A)

print(f"pool param details:")
print(f"initial_A: {initial_A}")
print(f"initial_A_time: {initial_A_time}")
print(f"A: {A}")
print(f"A_precise: {A_precise}")
print(f"A_precision: {A_precision}")

Disconnect:

In [ ]:
brownie.network.disconnect()